In [2]:
# Prepare paths of input images and target segmentation masks

In [3]:
import os
from IPython.display import Image, display
from keras.utils import load_img
from PIL import ImageOps
import keras
import numpy as np
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import io as tf_io
import random


In [4]:
# Dennis Code Cell for loading in dataset lung_image_sets_new (reduced to 3000 images)  to Google Colab

import zipfile
###import os

zip_file_path = '/content/sample_data/lung_image_sets_new.zip'
extract_folder = '/content/sample_data/lung_image_sets_new'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of zip file in the current directory
    zip_ref.extractall(extract_folder)

# Check if the extraction was successful
if os.path.isdir(extract_folder):
    print("Extraction successful.")
else:
    print("Extraction failed.")
#After running this code, you should be able to access the contents of the lung_image_sets_new directory for further processing.

Extraction successful.


In [5]:
# Dennis Code Cell for loading in dataset lung_image_sets_new (reduced to 3000 images)  to Google Colab

input_dir = extract_folder

In [16]:
# Dennis Code Cell for loading in dataset lung_image_sets_new (reduced to 3000 images)  to Google Colab

target_dir = "/content/sample_data/annotations_trimaps/"

In [11]:
#input_dir = "images/"
#target_dir = "annotations/trimaps/"
img_size = (160, 160)
num_classes = 3
batch_size = 32


In [12]:
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)


In [17]:
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)


In [18]:
print("Number of samples:", len(input_img_paths))

Number of samples: 0


In [19]:
for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)


What does one input image and corresponding segmentation mask look like?

In [20]:
# Display input image #7
#display(Image(filename=input_img_paths[9]))


IndexError: list index out of range

In [29]:
# Dennis Code Cell for loading in dataset lung_image_sets_new (reduced to 3000 images)  to Google Colab

len(input_img_paths)

# Display input image #7
#display(Image(filename=input_img_paths[9]))
#display(Image(filename=input_dir/lungaca1))

0

In [30]:
# Display auto-contrast version of corresponding target (per-pixel categories)
img = ImageOps.autocontrast(load_img(target_img_paths[9]))
display(img)


IndexError: list index out of range

Prepare dataset to load & vectorize batches of data

In [31]:
def get_dataset(
    batch_size,
    img_size,
    input_img_paths,
    target_img_paths,
    max_dataset_len=None,
):
    """Returns a TF Dataset."""


In [32]:
def load_img_masks(input_img_path, target_img_path):
        input_img = tf_io.read_file(input_img_path)
        input_img = tf_io.decode_png(input_img, channels=3)
        input_img = tf_image.resize(input_img, img_size)
        input_img = tf_image.convert_image_dtype(input_img, "float32")


In [34]:
        target_img = tf_io.read_file(target_img_path)
        target_img = tf_io.decode_png(target_img, channels=1)
        target_img = tf_image.resize(target_img, img_size, method="nearest")
        target_img = tf_image.convert_image_dtype(target_img, "uint8")


NameError: name 'target_img_path' is not defined

In [36]:
# Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
target_img -= 1
return input_img, target_img


NameError: name 'target_img' is not defined

In [37]:
    # For faster debugging, limit the size of data
    if max_dataset_len:
        input_img_paths = input_img_paths[:max_dataset_len]
        target_img_paths = target_img_paths[:max_dataset_len]
    dataset = tf_data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
    dataset = dataset.map(load_img_masks, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.batch(batch_size)


NameError: name 'max_dataset_len' is not defined

Prepare U-Net Xception-style model

In [38]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))


First half of the network: downsampling inputs

In [40]:
# Entry block

#x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
#x = layers.BatchNormalization()(x)
#x = layers.Activation("relu")(x)


NameError: name 'layers' is not defined

In [41]:
# Dennis Code Cell (debug)

# Entry block
from keras import layers

x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)


NameError: name 'inputs' is not defined

In [43]:
# Set aside residual
previous_block_activation = x


NameError: name 'x' is not defined

In [44]:
    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

NameError: name 'x' is not defined

In [45]:
        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual


NameError: name 'previous_block_activation' is not defined

Second half of the network: upsampling inputs

In [46]:
for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)


NameError: name 'x' is not defined

In [47]:
        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual


NameError: name 'previous_block_activation' is not defined

In [49]:
# Add a per-pixel classification layer
outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)


NameError: name 'x' is not defined

In [50]:
    # Define the model
    model = keras.Model(inputs, outputs)
    return model


NameError: name 'inputs' is not defined

In [52]:
# Build model
model = get_model(img_size, num_classes)
model.summary()
#model.info()

AttributeError: 'NoneType' object has no attribute 'info'

In [54]:
# Set aside a validation split

In [55]:
# Split our img paths into a training and a validation set
val_samples = 100      # resize from 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]


In [56]:
# Instantiate dataset for each split
# Limit input files in `max_dataset_len` for faster epoch training time.
# Remove the `max_dataset_len` arg when running with full dataset.
train_dataset = get_dataset(
    batch_size,
    img_size,
    train_input_img_paths,
    train_target_img_paths,
    max_dataset_len=1000,
)
valid_dataset = get_dataset(
    batch_size, img_size, val_input_img_paths, val_target_img_paths
)


Train the model

In [57]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(
    optimizer=keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy"
)

callbacks = [
    keras.callbacks.ModelCheckpoint("oxford_segmentation.keras", save_best_only=True)
]


AttributeError: 'NoneType' object has no attribute 'compile'

In [58]:
# Train the model, doing validation at the end of each epoch.
epochs = 50
model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=valid_dataset,
    callbacks=callbacks,
    verbose=2,
)


AttributeError: 'NoneType' object has no attribute 'fit'

Visualize predictions

In [59]:
# Generate predictions for all images in the validation set

val_dataset = get_dataset(
    batch_size, img_size, val_input_img_paths, val_target_img_paths
)
val_preds = model.predict(val_dataset)


#def display_mask(i):


SyntaxError: incomplete input (<ipython-input-59-963b49d138f6>, line 10)

In [61]:
   def display_mask(i):

    """Quick utility to display a model's prediction."""
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    display(img)


In [62]:
# Display results for validation image #10
i = 10


In [63]:
# Display input image
display(Image(filename=val_input_img_paths[i]))


IndexError: list index out of range

In [64]:
# Display ground-truth target mask
img = ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)


IndexError: list index out of range

In [65]:
# Display mask predicted by our model
display_mask(i)  # Note that the model only sees inputs at 150x150.


NameError: name 'val_preds' is not defined

In [ ]:
# Following is all Dennis Code Cells

# Need to transfer mask result to 0(lungaca), 1(lungn), or 2(lungscc)


# Patient Interface with Gradio - Image



In [67]:
# Create a Gradio interface
!pip install gradio
# Import Gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing i

In [68]:
# Create an instance of the Gradio Interface application function with the appropriate parameters.
input_img = gr.inputs.Image(shape=(150, 150))
target_img = gr.outputs.Image()
app = gr.Interface(fn=display_mask(), inputs=input_img, outputs=target_img, title='U-Net Image Segmentation')
# Launch the app
app.launch()

AttributeError: module 'gradio' has no attribute 'inputs'